In [15]:
import pandas as pd
import numpy as np
from scipy.stats import binom
import matplotlib.pyplot as plt
import utils
from utils import Plane, Handler
import visualize

### Ejercicio 1: Simulación de Montecarlo y Visualización

In [4]:
h = Handler(N_ITERS = 10_000)
results, planes = h.simulate()

In [3]:
histories = [p.history for p in planes]
visualize.visualize(histories,2.5, 18*60, AIRCRAFT_SVG_PATH="plane.svg")

Exited visualization..


### Ejercicio 2: Simulación con promedio de 1 arribo por hora
Si el promedio de arribos es 1 avión por hora y nosotros medimos minuto a minuto, $\lambda=\frac{1}{60}$

In [2]:
ej2 = Handler(N_ITERS = 10_000, LAMBDA = 1/60)
results, simulation_history = ej2.simulate()

### Ejercicio 3: Probabilidad de que lleguen 5 aviones en una hora

In [18]:
N_ITERS = 10_000
props_five = np.zeros(N_ITERS)
for i, sim in enumerate(simulation_history["simulations"]):
    first_appearances = [sim["plane_history"][i][0][0] for i in range(len(sim["plane_history"]))]
    groups = np.zeros((18,))
    more_than_five = 0
    for t0 in first_appearances:
        min_to_hour = int(t0 // 60)
        groups[min_to_hour] += 1
    more_than_five += (groups >= 5).astype(int).sum() / 18
    props_five[i] = more_than_five

print("Proba de que lleguen más de 5 aviones en una hora:", props_five.mean())
print("Error de estimación:", props_five.std())

Proba de que lleguen más de 5 aviones en una hora: 0.0035555555555555557
Error de estimación: 0.01397793675938558


Podríamos calcularlo de forma analítica de la siguiente manera:

$$
\begin{aligned}
    \text{Sea } X &= 
    \begin{cases}
        1 & \text{si arribó un avión en ese minuto} \\
        0 & \text{si no}
    \end{cases} \\[6pt]
    &X \sim \text{Be}\!\left(\tfrac{1}{60}\right) \\[6pt]
    \text{Definimos } S &= \sum_{i=1}^{60} X_i \sim Bin(60, \frac{1}{60}) \\[6pt]
    \text{Luego} \ Y &= "\text{proba de que lleguen 5 aviones en una hora}" \ = \ P(S \ge 5) \\[6pt]
    \text{Así, podemos calcular analíticamente: } P(S \ge 5) &= 1 - P(S \lt 5)
\end{aligned}
$$

In [16]:
rv = binom(60, 1/60)
1 - rv.cdf(4)

np.float64(0.0032833719684949303)

Así, vemos que el valor teórico de $P(S \ge 5)$  es 0.00328 $\approx$ 0.0033, lo cual no está lejos del valor estimado

### Ejercicio 4: Simulación de arribos para distintas tasas de arribo

In [ ]:
lambdas = [0.02, 0.1 , 0.2, 0.5, 1]
results = []

for l in lambdas:
    h = Handler(LAMBDA= l)
    res, p_hist = h.simulate()
    results.append({
        "lambda": l,
        "numeric_results": res,
        "plane_history": p_hist
    })

### Ejercicio 5: Simulación con interrupción de aterrizaje

In [ ]:
lambdas = [0.02, 0.1 , 0.2, 0.5, 1]
results = []
bounce_prob = 0.1
for l in lambdas:
    h = Handler(LAMBDA=l, PROP_BOUNCE=bounce_prob)
    res, p_hist = h.simulate()
    results.append({
        "lambda": l,
        "numeric_results": res,
        "plane_history": p_hist
    })

### Ejercicio 6: Simulación con cierre de media hora

In [ ]:
arrival_rate = 10 / 60
h = Handler(LAMBDA=arrival_rate, CLOSING_TIME=240) # closing_time = 240 -> el aeropuerto cierra a las 10:00 AM
res, p_hist = h.simulate()

### Ejercicio 7: Cambios para mejorar el funcionamiento del aeropuerto